# VacationPy
---

## Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-nera,64.5667,143.2000,71.10,33,28,4.50,RU,1718505878
1,1,albany,42.6001,-73.9662,61.21,68,3,4.88,US,1718505878
2,2,happy valley-goose bay,53.3168,-60.3315,53.91,58,75,8.05,CA,1718505878
3,3,gutierrez zamora,20.4500,-97.0833,80.78,84,0,2.91,MX,1718505878
4,4,waitangi,-43.9535,-176.5597,55.81,90,0,5.01,NZ,1718505878


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size='Humidity', 
    color='Humidity', alpha=0.6, frame_width=800, frame_height=600, 
    title="City Humidity"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 70) & (city_data_df['Max Temp'] <= 80) &
    (city_data_df['Wind Speed'] < 10) &
    (city_data_df['Cloudiness'] < 20)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,charters towers,-20.1000,146.2667,75.15,23,0,9.89,AU,1718505885
88,88,kievka,50.2619,71.5514,75.29,54,10,6.78,KZ,1718505671
93,93,togur,58.3631,82.8265,77.14,63,4,8.01,RU,1718505895
126,126,changji,44.0167,87.3167,74.19,43,0,2.24,CN,1718505901
128,128,port douglas,-16.4833,145.4667,74.86,62,5,5.70,AU,1718505901


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
40,charters towers,AU,-20.1000,146.2667,23,
88,kievka,KZ,50.2619,71.5514,54,
93,togur,RU,58.3631,82.8265,63,
126,changji,CN,44.0167,87.3167,43,
128,port douglas,AU,-16.4833,145.4667,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 5000  # e.g., 5000 meters
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1,  # to get the nearest hotel
    "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df.head())

Starting hotel search
charters towers - nearest hotel: No hotel found
kievka - nearest hotel: No hotel found
togur - nearest hotel: No hotel found
changji - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
kone - nearest hotel: Pacifik Appartels
shahr-e babak - nearest hotel: هتل شهرداری
craig - nearest hotel: Best Western Plus Deer Park Hotel And Suites
mandalgovi - nearest hotel: Б
la mira - nearest hotel: No hotel found
ocean city - nearest hotel: Watson's Regency Suites
khandyga - nearest hotel: Апельсин
sao vicente - nearest hotel: Pousada Vitória
taiynsha - nearest hotel: Гостиница "Орбита"
aguililla - nearest hotel: No hotel found
yeroham - nearest hotel: אירוס המדבר
yeppoon - nearest hotel: Blossom's on Seaspray
uzhur - nearest hotel: No hotel found
san felipe - nearest hotel: No hotel found
kyshtovka - nearest hotel: No hotel found
suez - nearest hotel: فندق لوسيندا
beichengqu - nearest hotel: 春江大酒店
al bawiti - nearest hotel: Old Oasis Ho

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:

hotel_df['Hover'] = hotel_df['City'] + ", " + hotel_df['Country'] + "\nHotel: " + hotel_df['Hotel Name']

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, size="Humidity", color="Humidity", 
    alpha=0.5, hover_cols=["Hover"], 
    tiles="OSM"
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hover)